In [ ]:
pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch.utils.data import Dataset, DataLoader
import random
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

#dir(models)

In [ ]:
#fix random seed
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
efficientNet = models.efficientnet_b0(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
print(efficientNet)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
efficientNet.classifier[1] = nn.Linear(1280, 11) #change output fc layer (to obtain the number of classes desired)

In [ ]:
efficientNet.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
#loading the saved file once again

dataset_train = torch.load('/content/drive/MyDrive/Vision/Dataset48/train_new.pt')

dataset_valid = torch.load('/content/drive/MyDrive/Vision/Validation_Test/validation48.pt')

dataset_test = torch.load('/content/drive/MyDrive/Vision/Validation_Test/test48.pt')

#dataset = np.load("/content/drive/MyDrive/Vision/Dataset48/features.npy")

In [ ]:
#list to encode/decode labels
indices_to_labels = ['arringatore', 'atena', 'atena_armata', 'demostene', 'dioniso', 'era_barberini', 'ercole', 'kouros_da_tenea', 'minerva_tritonia', 'poseidone', 'zeus']
labels_to_indices = {'arringatore': 0, 'atena': 1, 'atena_armata': 2, 'demostene': 3, 'dioniso': 4, 'era_barberini': 5, 'ercole': 6, 'kouros_da_tenea': 7, 'minerva_tritonia': 8, 'poseidone': 9, 'zeus': 10}

In [ ]:
train_labels = []

In [ ]:
#create labels

#train_labels = []
for lab in indices_to_labels:
  for i in range(0,144): #48
    train_labels.append(lab)

valid_labels = []
test_labels = []

for lab in indices_to_labels:
  for i in range(0,5):
    valid_labels.append(lab)
    test_labels.append(lab)

In [ ]:
#encode/decode labels functions

def encode_labels(labels, labels_to_indices):
  encoded = []
  for lab in labels:
    encoded.append(labels_to_indices[lab])
  return encoded

def decode_labels(encoded, indices_to_labels):
  labels = []
  for enc in encoded:
    labels.append(indices_to_labels[enc])
  return labels

def create_hot_encoder(encoded, num_classes):
  hot_encoder = torch.zeros(len(encoded), num_classes)
  c = 0
  for i in encoded:
    hot_encoder[c][i] = 1
    c += 1
  return hot_encoder


In [ ]:
train_labels = encode_labels(train_labels, labels_to_indices)
valid_labels = encode_labels(valid_labels, labels_to_indices)
test_labels = encode_labels(test_labels, labels_to_indices)

In [ ]:
hot_encoder_train = create_hot_encoder(train_labels, 11)
hot_encoder_valid = create_hot_encoder(valid_labels, 11)
hot_encoder_test = create_hot_encoder(test_labels, 11)

In [ ]:
data_train = {}
for i in range(0, len(dataset_train)):
  data_train[i]={'inputs': dataset_train[i], 'outputs': hot_encoder_train[i]}
  #data[i]={'inputs': torch.LongTensor(dataset[i]), 'outputs': hot_encoder[i]}

data_valid = {}
for i in range(0, len(dataset_valid)):
  data_valid[i]={'inputs': dataset_valid[i], 'outputs': hot_encoder_valid[i]}

data_test = {}
for i in range(0, len(dataset_test)):
  data_test[i]={'inputs': dataset_test[i], 'outputs': hot_encoder_test[i]}


In [ ]:
train_dataloader = DataLoader(data_train, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(data_valid, batch_size=8, shuffle=False)
test_dataloader = DataLoader(data_test, batch_size=8, shuffle=False)

In [ ]:
def train_model(model, train_dataloader, criterion, optimizer, valid_dataloader=None, num_epochs=25, is_inception=False):
    since = time.time()
    since2 = since

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('----------')
        
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for data in train_dataloader:
          inputs = torch.squeeze(data['inputs'])
          labels = data['outputs']
          #inputs = inputs.to(device)
          #labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          outputs = model(inputs)
          loss = criterion(outputs, labels)

          _, preds = torch.max(outputs, 1)

            # backward + optimize only if in training phase
          loss.backward()
          optimizer.step()

            # statistics
          running_loss += loss.item() * inputs.size(0)
          _, correct = torch.max(labels, 1)
          running_corrects += torch.sum(preds == correct)

          epoch_loss = running_loss / len(train_dataloader.dataset)
          

        print()
        print('Total Epoch loss: ', epoch_loss)
        print('Total correct classified: ', running_corrects, ' total classified: ', len(train_dataloader)*8)
        time_elapsed = time.time() - since2
        since2 = time.time()
        print('Time to complete epochs {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

        if valid_dataloader:
          print('Validation Results:')
          evaluate_model (model, valid_dataloader)

        running_loss = 0.0
        running_corrects = 0

    print('-----end-----')
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return model



def evaluate_model(model, dataloaders, top_three=False):
  running_corrects = 0
  for data in dataloaders:
    inputs = torch.squeeze(data['inputs'])
    labels = data['outputs']
    outputs = model(inputs)

    _, index = torch.max(outputs, 1)

    percentage = torch.nn.functional.softmax(outputs, dim=1)[0] * 100

    #print('The class assigned is: ', index[0], ' with the percentage: ', percentage[index[0]].item())

    if top_three:
      _, indices = torch.sort(outputs, descending=True)
      [(idx, percentage[idx].item()) for idx in indices[0][:3]]
      print(index[0], percentage[index[0]].item())

    _, correct = torch.max(labels, 1)
    running_corrects += torch.sum(index == correct)

        
  print('Total correct classified: ', running_corrects, ' total classified: ', len(dataloaders)*8)


In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(efficientNet.parameters())

In [ ]:
efficientNet = train_model(efficientNet, train_dataloader, loss_function, optimizer, valid_dataloader, 5)

Epoch 0/4
----------

Total Epoch loss:  0.2582861603850045
Total correct classified:  tensor(2950)  total classified:  3168
Time to complete epochs 17m 1s
Validation Results:
Total correct classified:  tensor(23)  total classified:  56
Epoch 1/4
----------

Total Epoch loss:  0.07898411637312036
Total correct classified:  tensor(3103)  total classified:  3168
Time to complete epochs 14m 47s
Validation Results:
Total correct classified:  tensor(21)  total classified:  56
Epoch 2/4
----------

Total Epoch loss:  0.07996203483537212
Total correct classified:  tensor(3103)  total classified:  3168
Time to complete epochs 14m 44s
Validation Results:
Total correct classified:  tensor(30)  total classified:  56
Epoch 3/4
----------

Total Epoch loss:  0.03164313369662256
Total correct classified:  tensor(3141)  total classified:  3168
Time to complete epochs 14m 44s
Validation Results:
Total correct classified:  tensor(24)  total classified:  56
Epoch 4/4
----------

Total Epoch loss:  0.070

In [ ]:
#save
PATH2 = '/content/drive/MyDrive/Vision/Dataset48/EfficientNet_new.pt'
torch.save(efficientNet, PATH2)

In [ ]:
#load
PATH2 = '/content/drive/MyDrive/Vision/Dataset48/EfficientNet_new.pt'
model = torch.load(PATH2)
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
evaluate_model(model, test_dataloader)

Total correct classified:  tensor(26)  total classified:  56


Prediction

In [ ]:
t = torch.load('/content/drive/MyDrive/Vision/Try/prova1.pt')
batch_t = torch.unsqueeze(t, 0)
out = model(batch_t)
print(out.shape)

torch.Size([1, 11])


In [ ]:
_, index = torch.max(out, 1)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(index[0], percentage[index[0]].item())

tensor(0) 99.99781799316406


In [ ]:
_, indices = torch.sort(out, descending=True)
[(idx, percentage[idx].item()) for idx in indices[0][:5]]

[(tensor(0), 99.99781799316406),
 (tensor(3), 0.002177407266572118),
 (tensor(4), 5.234362561168382e-07),
 (tensor(5), 2.1240138892153482e-07),
 (tensor(2), 8.32945872275559e-08)]